In [7]:
import cv2
import numpy as np
from keras_squeezenet import SqueezeNet
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.layers import Activation, Dropout, Convolution2D, GlobalAveragePooling2D
from keras.models import Sequential
import tensorflow as tf
import os

In [8]:
IMG_SAVE_PATH = 'image_data'

CLASS_MAP = {
    "rock": 0,
    "paper": 1,
    "scissors": 2,
    "none": 3
}


In [9]:
NUM_CLASSES = len(CLASS_MAP)

In [10]:
def mapper(val):
    return CLASS_MAP[val]

In [11]:
def get_model():
    model = Sequential([
        SqueezeNet(input_shape=(227, 227, 3), include_top=False),
        Dropout(0.5),
        Convolution2D(NUM_CLASSES, (1, 1), padding='valid'),
        Activation('relu'),
        GlobalAveragePooling2D(),
        Activation('softmax')
    ])
    return model

In [12]:
dataset = []
for directory in os.listdir(IMG_SAVE_PATH):
    path = os.path.join(IMG_SAVE_PATH, directory)
    if not os.path.isdir(path):
        continue
    for item in os.listdir(path):
        # to make sure no hidden files get in our way
        if item.startswith("."):
            continue
        img = cv2.imread(os.path.join(path, item))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (227, 227))
        dataset.append([img, directory])

In [13]:
data, labels = zip(*dataset)
labels = list(map(mapper, labels))

In [14]:
labels = np_utils.to_categorical(labels)

In [15]:
model = get_model()
model.compile(
    optimizer=Adam(lr=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [ ]:
model.fit(np.array(data), np.array(labels), epochs=10)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
12000/12000 [==============================] - 619s 52ms/step - loss: 0.0756 - acc: 0.9723
Epoch 2/10
 9280/12000 [======================>.......] - ETA: 2:10 - loss: 0.0031 - acc: 0.9994

In [ ]:
model.save("rock-paper-scissors-model.h5")